# Rechunker Tutorial

This tutorial notebook explains how to use rechunker with real datasets. We will also use xarray to make some things easier and prettier, but we note that xarray is not a dependency for rechunker.

## Toy Example

### Create Example Data

Here we load one of xarray's tutorial datasets and write it to Zarr. This is not actually a big dataset, so rechunker is not really needed here. But it's a convenient example.

In [ ]:
import xarray as xr
xr.set_options(display_style='text')
import zarr
import dask.array as dsa

ds = xr.tutorial.open_dataset("air_temperature")
# create initial chunk structure
ds = ds.chunk({'time': 100})
ds.air.encoding = {} # helps when writing to zarr
ds

We can examine the chunk structure of the data variable using Dask's pretty Array repr.

In [ ]:
ds.air.data

In [ ]:
! rm -rf *.zarr # clean up any existing temporary data
ds.to_zarr('air_temperature.zarr')

Now we open up a Zarr Group and Array that we will use as inputs to rechunker.

In [ ]:
source_group = zarr.open('air_temperature.zarr')
print(source_group.tree())

In [ ]:
source_array = source_group['air']
source_array.info

### Rechunk a single Array

The original array has chunks of (100, 25, 53). Let's rechunk it to be contiguous in time, but chunked in space.
We specify a small value of `max_mem` in order to force rechunker to create an intermediate dataset. We also have to specify a place to store the final and intermediate data.

We use the [rechunk](api.rst#rechunker.rechunk) function, which returns a [Rechunked](api.rst#rechunker.Rechunked) object.

In [ ]:
from rechunker import rechunk

target_chunks = (2920, 25, 1)
max_mem = '1MB'

target_store = 'air_rechunked.zarr'
temp_store = 'air_rechunked-tmp.zarr'

array_plan = rechunk(source_array, target_chunks, max_mem, target_store, temp_store=temp_store)
array_plan

Since this array has dimensions, we can also specify the chunks using a dictionary syntax.

In [ ]:
target_chunks_dict = {'time': 2920, 'lat': 25, 'lon': 1}

# need to remove the existing stores or it won't work
!rm -rf air_rechunked.zarr air_rechunked-tmp.zarr
array_plan = rechunk(source_array, target_chunks_dict, max_mem, target_store, temp_store=temp_store)
array_plan

The `array_plan` is a `Rechunked` object.
It has not actually performed the rechunking yet.
To do this, we need to call the `execute` method.
This will use Dask to perform the rechunking.

In [ ]:
result = array_plan.execute()
result.chunks

By default, Dask will use the multi-threaded scheduler.
Since rechunking can take a long time, we might want to use a progress bar.

In [ ]:
from dask.diagnostics import ProgressBar

with ProgressBar():
    array_plan.execute()

If we create a distributed cluster, then rechunker will use that when it executes.

In [ ]:
from dask.distributed import Client, LocalCluster, progress

cluster = LocalCluster()
client = Client(cluster)

Now that it is written to disk, we can open the rechunked array however we please. Using Zarr...

In [ ]:
target_array = zarr.open('air_rechunked.zarr')
target_array

...or Dask

In [ ]:
target_array_dask = dsa.from_zarr('air_rechunked.zarr')
target_array_dask

### Rechunk a Group

In the example above, we only rechunked a single array.
We can open it with Dask, but not Xarray, because it doesn't contain any coordinates or metadata.

Rechunker also supports rechunking entire groups.
In this case, `target_chunks` must be a dictionary.

In [ ]:
target_chunks = {
    'air': {'time': 2920, 'lat': 25, 'lon': 1},
    'time': None, # don't rechunk this array
    'lon': None,
    'lat': None,
}
max_mem = '1MB'

target_store = 'group_rechunked.zarr'
temp_store = 'group_rechunked-tmp.zarr'

# need to remove the existing stores or it won't work
!rm -rf group_rechunked.zarr group_rechunked-tmp.zarr
array_plan = rechunk(source_group, target_chunks, max_mem, target_store, temp_store=temp_store)
array_plan

In [ ]:
array_plan.execute()

Now that we have written a group, we can open it back up with Xarray.

In [ ]:
xr.open_zarr('group_rechunked.zarr')

Often groups have many variables sharing all or a subset of dimensions. In the common case that a given dimension should have equivalent chunks in each variable that contains it, chunks can be provided as a simpler dictionary, mapping dimension names to chunksize.

In [ ]:
# extend the dataset with some more variables
ds_complex = ds
ds_complex['air_slice'] = ds.air.isel(lat=10)
ds_complex['air_timeseries'] = ds.air.isel(lat=10, lon=10)
ds_complex

target_chunks = {'time': 2920, 'lat': 25, 'lon': 1}
max_mem = '1MB'

target_store = 'group_complex_rechunked.zarr'
temp_store = 'group_complex_rechunked-tmp.zarr'

# need to remove the existing stores or it won't work
!rm -rf group_complex_rechunked.zarr group_complex_rechunked-tmp.zarr

# rechunk directly from dataset this time
array_plan = rechunk(ds_complex, target_chunks, max_mem, target_store, temp_store=temp_store)
array_plan

In [ ]:
array_plan.execute()

In [ ]:
xr.open_zarr('group_complex_rechunked.zarr')

Note that all the variables now have the same time chunks. Other dimensions (if they exist) also have consistent chunks. 

## Cloud Example

In this example we use real data from Pangeo's [Cloud Data Catalog](http://catalog.pangeo.io/).
This dataset is stored in Google Cloud Storage.
We also use a [Dask Gateway](https://gateway.dask.org/) distributed cluster to scale up our processing.
This part of the tutorial won't work for you unless you are in a [Pangeo Cloud](http://pangeo.io/cloud.html) environment or binder.

In [ ]:
from dask_gateway import GatewayCluster
cluster = GatewayCluster()
cluster.scale(20)
cluster

In [ ]:
from dask.distributed import Client
client = Client(cluster)
client

In [ ]:
import gcsfs
# a zarr group lives here
url = 'gs://pangeo-cmems-duacs'
gcs = gcsfs.GCSFileSystem(requester_pays=True)
source_store = gcs.get_mapper(url)

### Open Zarr Array

In [ ]:
group = zarr.open_consolidated(source_store, mode='r')
source_array = group['sla']
source_array

In [ ]:
source_array.chunks

### Make a Rechunking Plan

In [ ]:
max_mem = '1GB'
target_chunks = (8901, 72, 72)
# you must have write access to this location
store_tmp = gcs.get_mapper('pangeo-scratch/rabernat/rechunker_demo/temp.zarr')
store_target = gcs.get_mapper('pangeo-scratch/rabernat/rechunker_demo/target.zarr')
r = rechunk(source_array, target_chunks, max_mem,
                      store_target, temp_store=store_tmp)
r

### Execute the Plan

In [ ]:
result = r.execute()
result

In [ ]:
dsa.from_zarr(result)